# Base parameters and combinations with QR

In the following, the QR decomposition is used to find columns in a matrix (e.g. dynamics regressor) that are independent and therefore are a choice of a basis for the identifiable subspace. It is then determined which linear combinations of the other dependent columns are forming the base columns/parameters in each of those. This is also necessary for the std essential parameters.

In [48]:
import numpy as np; np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

In [259]:
a = np.array([[1.0, 2.0,  .0,  .0, 2.0],
              [1.0, 2.0,  .0,  .0, 1.0],
              [ .0,  .0, 1.0, 5.0,  .0],
              [ .0,  .0, 1.0, 5.0,  .0],
              [2.0, 4.0,  .0,  .0,  .0],
              [ .0,  .0, 0.5, 2.5,  .0]])
print(a.shape)
r = np.linalg.matrix_rank(a)
r

(6, 5)


3

Test Matrix $a$ is rank deficient (rank $r<m$). The rank 3 means there are 3 sets of dependent columns, each with 1 or more columns in it. The columns $0,1$ and $2,3$ are linearly dependent. Column 4 is linearly independent from all other columns. Finding independent columns will give a single representant of each set, while the choice is non-unique if any set has more than one element. If a column is independent, there can be other columns that are dependent on it but it can also be a single element in its set.

$$a\cdot x = t$$

In [291]:
x = np.array([1.0,2.0,0.838,4.192,4.5])  #param
t = a.dot(x)                         #result
t

array([ 14.000,  9.500,  21.798,  21.798,  10.000,  10.899])

## Sets of dependent columns

Assuming $m \times n$ Matrix $a$ with $m>=n$.

In [292]:
import scipy.linalg as sla
Qp,Rp,P = sla.qr(a, pivoting=True, check_finite=True)
print(Rp.diagonal())
print(P)
print()
Q,R = np.linalg.qr(a, mode='complete')
print(R.diagonal())

[-7.500 -4.899  1.871 -0.000 -0.000]
[3 1 4 0 2]

[-2.449  0.000 -1.500  0.000 -0.894]


Find selection of columns that are linearly independent to each other. For non-pivoting QR, the columns in R with diagonal elements that are not zero are independent. For pivoting QR this does not work. However, as there are as many independent columns as the rank of the matrix, taking the first rank columns of the permutation also gives the independent column indices but in different order. The choices are not the same but from the same sets of dependent columns.

In [293]:
tol = 1e-6
print(R.diagonal())
indepsQR = np.where(np.abs(R.diagonal()) > tol)[0]
print(indepsQR)

print("rank: {}".format(np.where(np.abs(Rp.diagonal()) > tol)[0].size ))   #rank with pivoting QR
indepsQRP = P[0:r]
print(indepsQRP)


[-2.449  0.000 -1.500  0.000 -0.894]
[0 2 4]
rank: 3
[3 1 4]


In [294]:
#create proper permutation matrix
Pp = np.zeros((P.size, P.size))
for i in P:
    Pp[i,P[i]]=1

In [295]:
#Gautier 1990, 3-4: linear relationships
#W1W2 = a.dot(Pp)
#W1 = W1W2[:,0:indeps.size]
#W2 = W1W2[:,indeps.size:a.shape[1]]
#Q,R = np.linalg.qr(W1W2)
#print(Q)
#print(R.diagonal())
#corresponds to QR with pivoting
#print(Qp)
#print(Rp.diagonal())

#dependent columns W1 as linear combinations of independent columns W1
R1 = Rp[0:indepsQRP.size,0:indepsQRP.size]
R2 = Rp[0:indepsQRP.size,indepsQRP.size:]
linDeps = np.linalg.inv(R1).dot(R2)
print(linDeps)

[[ 0.000  0.200]
 [ 0.500  0.000]
 [ 0.000 -0.000]]


In [296]:
print(P)
for i in range(0,linDeps.shape[0]):
    for j in range(0,linDeps.shape[1]):
        if np.abs(linDeps[i,j]) > 0.01:
            dep_org_col = P[indepsQRP.size+i]
            indep_org_col = P[j]

            print(
                '''col {} in W2(col {} in a) is a linear combination of col {} in W1 (col {} in a) with factor {}'''\
                   .format(i, dep_org_col, j, indep_org_col, linDeps[i,j]))

[3 1 4 0 2]
col 0 in W2(col 0 in a) is a linear combination of col 1 in W1 (col 1 in a) with factor 0.2
col 1 in W2(col 2 in a) is a linear combination of col 0 in W1 (col 3 in a) with factor 0.5000000000000002


## Getting the base projection matrix

The first $r$ orthogonal columns of $Q$ form a basis $B$ of the column space of $A$ in case a is a square matrix. In the rectangular case, the QR decomposition of $A^T$ seems to be possible but maybe not always solvable?. 
$B$ can be used to project the matrix $A$ with regard to a vector $v$ to a matrix $A_\text{base}$ with regard to a vector $v_\text{base}$.

In [305]:
Qpt,Rpt,Pt = sla.qr(a.T, pivoting=True, mode='economic')
print(Qpt)
B = -Qpt[:, 0:r]
print(B)
#B = sla.orth(a)[0:a.shape[1], :]
#print(B)

[[ 0.000 -0.447 -0.000  0.894  0.000]
 [-0.000 -0.894  0.000 -0.447 -0.000]
 [-0.196 -0.000 -0.000  0.000 -0.981]
 [-0.981 -0.000  0.000  0.000  0.196]
 [-0.000 -0.000 -1.000  0.000  0.000]]
[[-0.000  0.447  0.000]
 [ 0.000  0.894 -0.000]
 [ 0.196  0.000  0.000]
 [ 0.981  0.000 -0.000]
 [ 0.000  0.000  1.000]]


$Pp^T XB = [XB1 ; XB2] = [X1 + X1^{-1}R2 X2;0]$

In [298]:
print(P)
Px = Pp.dot(x)
print('x:\t', x)
x1 = Px[0:r]
x2 = Px[r:]
print ('x1, x2:\t', x1, x2)
xb1 = x1 + np.linalg.inv(R1).dot(R2.dot(x2))
print (xb1)

[3 1 4 0 2]
x:	 [ 1.000  2.000  0.838  4.192  4.500]
x1, x2:	 [ 4.192  2.000  4.500] [ 1.000  0.838]
[ 4.360  2.500  4.500]


In [299]:
a_base = a.dot(B)
x_base = B.T.dot(x)
print('x_base:\t', x_base)
print("deps:")
for i in range(0, len(x_base)):
    print("{}, ".format(indepsQRP[i], deps[i]))
t_base = a_base.dot(x_base)
print('t:\t', t)
print('t_base:\t', t_base)
print('x:\t', x)
print ('x*:\t', B.dot(x_base))    #should give same x as in standard space if x was fully in base space already

# test nullspace
print(a.dot(np.eye(5) - B.dot(B.T)))

x_base:	 [ 4.275  2.236  4.500]
deps:
3, 
1, 
4, 
t:	 [ 14.000  9.500  21.798  21.798  10.000  10.899]
t_base:	 [ 14.000  9.500  21.798  21.798  10.000  10.899]
x:	 [ 1.000  2.000  0.838  4.192  4.500]
x*:	 [ 1.000  2.000  0.838  4.192  4.500]
[[-0.000  0.000 -0.000 -0.000  0.000]
 [-0.000  0.000 -0.000 -0.000  0.000]
 [-0.000 -0.000 -0.000 -0.000  0.000]
 [-0.000 -0.000 -0.000 -0.000  0.000]
 [ 0.000  0.000 -0.000 -0.000  0.000]
 [-0.000 -0.000 -0.000 -0.000  0.000]]


In [300]:
deps = np.linalg.inv(Rpt[0:r, 0:r]).dot(Rpt[0:r, r:])
print(deps)

[[ 1.000  0.000  0.500]
 [ 0.000  0.250  0.000]
 [ 0.000  0.500  0.000]]


### Base Projection with SVD

In [307]:
U, s, Vh = sla.svd(a, full_matrices=False)
print(U)
print(Vh)
np.sum(s>0.001)

[[ 0.000 -0.472 -0.766  0.436  0.000]
 [ 0.000 -0.429 -0.233 -0.873  0.000]
 [-0.667  0.000  0.000  0.000  0.596]
 [-0.667  0.000  0.000  0.000 -0.745]
 [ 0.000 -0.771  0.599  0.218  0.000]
 [-0.333  0.000  0.000  0.000  0.298]]
[[-0.000 -0.000 -0.196 -0.981  0.000]
 [-0.434 -0.867  0.000  0.000 -0.244]
 [ 0.109  0.218  0.000  0.000 -0.970]
 [-0.894  0.447  0.000  0.000 -0.000]
 [-0.000 -0.000 -0.981  0.196  0.000]]


3

In [302]:
# QR base for comparison
B

array([[-0.000,  0.447,  0.000],
       [ 0.000,  0.894, -0.000],
       [ 0.196,  0.000,  0.000],
       [ 0.981,  0.000, -0.000],
       [ 0.000,  0.000,  1.000]])

In [306]:
B_svd = -Vh.T[:, 0:r]
B = B_svd
B

array([[ 0.000,  0.434, -0.109],
       [ 0.000,  0.867, -0.218],
       [ 0.196, -0.000, -0.000],
       [ 0.981, -0.000, -0.000],
       [-0.000,  0.244,  0.970]])